In [ ]:
from nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='../nuscenes', verbose=True)


In [ ]:
from matplotlib.patches import Rectangle
Rectangle(xy=(0,0), width=10, height=10, angle=0.0, rotation_point='xy')

In [ ]:
nusc.render_sample('fd8420396768425eabec9bdddf7e64b6')

In [ ]:
# import quaternion to euler angle from scipy
angles = []
from scipy.spatial.transform import Rotation as R
anns = nusc.get('sample', "2f92c83f32aa410b93d58ea9948aba74")['anns']
count = 0
for ann in anns:
    entry = nusc.get('sample_annotation', ann)
    # if entry['category_name'] != 'vehicle.car' or entry['num_lidar_pts'] == 0 or entry['num_radar_pts'] == 0:
    #     continue
    # convert quaternion to euler angle
    r = R.from_quat(entry['rotation'], scalar_first=True)
    angles.append(r.as_euler('xyz', degrees=False)[-1])
    print(r.as_euler('xyz', degrees=False))
    count += 1
print(count)


In [ ]:
nusc.get_box(anns[0])


In [ ]:
# Loop through all sample_data and verify if LIDAR_TOP file exists
import os
from tqdm import tqdm
count = 0
for sample in tqdm(nusc.sample):
    lidar_token = sample['data']['LIDAR_TOP']
    lidar_path = nusc.get_sample_data_path(lidar_token)
    # try to open the file
    if not os.path.exists(lidar_path):
        count += 1
        print(f"File {lidar_path} does not exist")
        continue
print(f"{count} files do not exist")

In [ ]:
import pickle
import json

# Load the Pickle file
with open('/home/vxm240030/CenterPoint/work_dirs/nusc_centerpoint_pp_02voxel_two_pfn_10sweep/train/version_12/prediction.pkl', 'rb') as f:
    data = pickle.load(f)

# Convert the data to JSON format and save to a JSON file
x = {}
x['results'] = data
with open('/home/vxm240030/CenterPoint/work_dirs/nusc_centerpoint_pp_02voxel_two_pfn_10sweep/train/version_12/prediction.json', 'w') as f:
    json.dump(x, f)

# x['results'][0]


In [1]:
from nuscenes import NuScenes
from nuscenes.eval.detection.evaluate import DetectionEval, DetectionConfig
from nuscenes.eval.common.config import config_factory
import json
cfg = config_factory('detection_cvpr_2019')
# with open('cfg.json') as f:
#     cfg = DetectionConfig.deserialize(json.load(f))
nusc = NuScenes(version='v1.0-trainval', dataroot='data/nuScenes', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 17.430 seconds.
Reverse indexing ...
Done reverse indexing in 4.3 seconds.


In [5]:
eval = DetectionEval(nusc, cfg, './work_dirs/nusc_onestage_custom_test/train/version_3/predictions.json', 'val', 
                     './work_dirs/nusc_onestage_custom_test/train/version_3/nusclogs')

Initializing nuScenes detection evaluation
Loaded results from ./work_dirs/nusc_onestage_custom_test/train/version_3/predictions.json. Found detections for 6019 samples.
Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 6019/6019 [00:04<00:00, 1246.98it/s]


Loaded ground truth annotations for 6019 samples.
Filtering predictions
=> Original number of boxes: 59170
=> After distance based filtering: 0
=> After LIDAR and RADAR points based filtering: 0
=> After bike rack filtering: 0
Filtering ground truth annotations
=> Original number of boxes: 187528
=> After distance based filtering: 134565
=> After LIDAR and RADAR points based filtering: 121871
=> After bike rack filtering: 121861


In [ ]:
eval.main(plot_examples=5)